In [1]:
import pandas as pd
import pyspark.pandas as ps
import requests
import json

from sqlalchemy import create_engine

from pyspark.sql.functions import pandas_udf, PandasUDFType, max, col, countDistinct, when, rank, lit
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType
from pyspark.sql.window import Window

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from imblearn.under_sampling import ClusterCentroids



# from backend_spark.doris_common.doris_client import DorisClient
from functools import reduce
from pyspark.sql import DataFrame
from typing import Optional

In [7]:
#####################################################################################
#######################################解析SQL########################################
#####################################################################################

In [22]:
# ## 这里使用doris数据库连接
# user ="root"
# host = "10.52.199.81"
# password = "Nexchip%40123"
# db = "etl"
# port = 9030

# engine = create_engine("mysql+pymysql://{user}:{password}@{host}:{port}/{db}".format(user = user,
#                                                                                     password = password,
#                                                                                     host = host,
#                                                                                     port = port,
#                                                                                     db = db))

# df1_pandas = pd.read_sql_query("SELECT * FROM etl.DWD_POC_CASE_FD_UVA_DATA_CASE1_PROCESSED1", engine)   

In [2]:
df1_pandas = pd.read_csv("DWD_POC_CASE_FD_UVA_DATA_CASE1_PROCESSED1.csv")
df1_pandas

,WAFER_ID,TOOL_ID,RUN_ID,EQP_NAME,PRODUCT_ID,PRODG1,TOOL_NAME,LOT_ID,RECIPE_NAME,OPER_NO,START_TIME,parametric_name,CASE_INFO,STATUS,STATISTIC_RESULT,label
0,NGE186-06,11341,149769,EKT72,AFKN2J01N.0U01,L11CD02A,EKT72_PM1,NGE186.000,NEW-DRM/P1/110NM/PFKN0S0D1F1A,1F.EEK10,2023-06-16 02:12:04,PROCESS_GAS_5_CHF3#WINDOW_1#SUM,2023-06-16,NORMAL,123907.6000,0
1,NGE186-06,11341,149769,EKT72,AFKN2J01N.0U01,L11CD02A,EKT72_PM1,NGE186.000,NEW-DRM/P1/110NM/PFKN0S0D1F1A,1F.EEK10,2023-06-16 02:12:04,PROCESS_GAS_4_CH2F2#WINDOW_1#SUM,2023-06-16,NORMAL,0.0000,0
2,NGE186-06,11341,149769,EKT72,AFKN2J01N.0U01,L11CD02A,EKT72_PM1,NGE186.000,NEW-DRM/P1/110NM/PFKN0S0D1F1A,1F.EEK10,2023-06-16 02:12:04,PROCESS_GAS_6_CF4#WINDOW_1#SUM,2023-06-16,NORMAL,0.0000,0
3,NGE186-06,11341,149769,EKT72,AFKN2J01N.0U01,L11CD02A,EKT72_PM1,NGE186.000,NEW-DRM/P1/110NM/PFKN0S0D1F1A,1F.EEK10,2023-06-16 02:12:04,ESC_CURRENT#AOTU_STEP_2#MEAN,2023-06-16,NORMAL,62.0000,0
4,NGE186-06,11341,149769,EKT72,AFKN2J01N.0U01,L11CD02A,EKT72_PM1,NGE186.000,NEW-DRM/P1/110NM/PFKN0S0D1F1A,1F.EEK10,2023-06-16 02:12:04,PROCESS_GAS_2_C4F8#AOTU_STEP_2#MEAN,2023-06-16,NORMAL,0.0000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550906,NGG239-19,11341,153513,EKT72,AMKNS301N.0B01,L11CB14A,EKT72_PM1,NGG239.000,NEW-DRM/P1/110NM/PFKN9F0D1F1A,1F.EEK10,2023-06-30 12:01:39,ESC_VOLTAGE#AOTU_STEP_2#MEAN,2023-06-30,NORMAL,2503.0000,0
550907,NGG239-19,11341,153513,EKT72,AMKNS301N.0B01,L11CB14A,EKT72_PM1,NGG239.000,NEW-DRM/P1/110NM/PFKN9F0D1F1A,1F.EEK10,2023-06-30 12:01:39,PROCESS_GAS_11_N2#AOTU_STEP_2#MEAN,2023-06-30,NORMAL,0.0000,0
550908,NGG239-19,11341,153513,EKT72,AMKNS301N.0B01,L11CB14A,EKT72_PM1,NGG239.000,NEW-DRM/P1/110NM/PFKN9F0D1F1A,1F.EEK10,2023-06-30 12:01:39,PROCESS_GAS_4_CH2F2#AOTU_STEP_2#MEAN,2023-06-30,NORMAL,0.0000,0
550909,NGG239-19,11341,153513,EKT72,AMKNS301N.0B01,L11CB14A,EKT72_PM1,NGG239.000,NEW-DRM/P1/110NM/PFKN9F0D1F1A,1F.EEK10,2023-06-30 12:01:39,PROCESS_GAS_7_AR#AOTU_STEP_2#MEAN,2023-06-30,NORMAL,199.9009,0


In [3]:
df1 = ps.from_pandas(df1_pandas).to_spark()

/usr/local/spark/python/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/usr/local/spark/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/spark/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [4]:
df1.count()

550911

In [5]:
############################################
######## 1. 客户只定义了bad_wafer = []是什么  ########
############################################
# 将传进来的BAD_WAFER, 用 | 连接起来，
# F.col('WAFER_ID').like('NDJ065%') | F.col('WAFER_ID').like('NDJ067%') 作为条件传入增加label
# 同时将isin模式也作为条件传入增加label

def get_label_single(df, bad_wafer):
    like_conditions = [f"col('WAFER_ID').like('{bad}')" for bad in bad_wafer]
    all_like_conditions = " | ".join(like_conditions)
    isin_conditions = "col('WAFER_ID').isin(bad_wafer)"
    df = df.withColumn('label', 
                when( eval(all_like_conditions) | eval(isin_conditions), int(1)).otherwise(int(0)))
    return df


############################################
## 2. 客户定义了bad_wafer = [] 和 good_wafer = []######
############################################
# 将传进来的BAD_WAFER, 用 | 连接起来，
# 将传进来的GOOD_WAFER, 也用 | 连接起来，
# 同时将isin模式也作为条件传入增加label

def get_label_double(df, bad_wafer, good_wafer):
    good_like_conditions = [f"col('WAFER_ID').like('{good}')" for good in good_wafer]
    all_good_like_conditions = " | ".join(good_like_conditions)
    good_isin_conditions = "col('WAFER_ID').isin(good_wafer)"

    bad_like_conditions = [f"col('WAFER_ID').like('{bad}')" for bad in bad_wafer]
    all_bad_like_conditions = " | ".join(bad_like_conditions)
    bad_isin_conditions = "col('WAFER_ID').isin(bad_wafer)"

    df = df.withColumn('label',  when(eval(all_good_like_conditions) | eval(good_isin_conditions), int(0)).when(eval(all_bad_like_conditions) | eval(bad_isin_conditions), int(1)).otherwise(222333))
    df = df.filter(df['label'] != int(222333))
    return df

In [5]:
# small1.csv
# good = ["NBX221-13","NBX220-06","NBX272-19","NBX219-08"]
# bad  = ["NBX265-05","NBX265-06","NBX265-07","NBX265-08",
#         "NBX293-06","NBX293-07","NAZ998-21","NBX220-20"]

# df1 = get_label_double(df1, bad, good)

# small2.csv
# good = ["NBX221-13","NBX220-06","NBX272-19","NBX219-08"]
# bad  = ["NBX265-05","NBX265-06","NBX265-07","NBX265-08","NBX293-06","NBX293-07","NAZ998-21","NBX220-20"]

# df1 = get_label_double(df1, bad, good)
# df1.count()

In [6]:
bad_wafer_num = df1.filter("label == 1").select('WAFER_ID').distinct().count()
good_wafer_num = df1.filter("label == 0").select('WAFER_ID').distinct().count()
print(bad_wafer_num)
print(good_wafer_num)

2685
3409


In [7]:
# df1.filter("label == 1").select('WAFER_ID').distinct().count()

In [7]:
############################################################################
##################################FDC数据预处理###############################
############################################################################
def _pre_process(df):
    """
    param df: 从数据库中读取出来的某个CASE数据
    return: 数据预处理，后面要根据实际情况统一添加
    """
    try:
        # 只选出会用到的列
        df = df.select('WAFER_ID', 'TOOL_ID', 'RUN_ID', 'EQP_NAME', 'PRODUCT_ID', 'PRODG1', 'TOOL_NAME',
                       'OPER_NO', 'parametric_name', 'STATISTIC_RESULT', 'label')
        # 剔除NA值
        df = df.filter(col('STATISTIC_RESULT').isNotNull())
        # 按照所有的行进行去重
        df1 = df.dropDuplicates()
        # 选最新的RUN
        df2 = df1.groupBy('WAFER_ID', 'OPER_NO', 'TOOL_ID').agg(max('RUN_ID').alias('RUN_ID'))
        df_run = df1.join(df2.dropDuplicates(subset=['WAFER_ID', 'OPER_NO', 'TOOL_ID', 'RUN_ID']),
                          on=['WAFER_ID', 'OPER_NO', 'TOOL_ID', 'RUN_ID'], how='inner')
        return df_run
    except Exception as e:
        return None



def commonality_analysis(df_run, grpby_list):
    """
    param df_run: 数据预处理后的数据
    return: 共性分析后的结果， 返回bad wafer前十的组合
    """
    try:
        grps = (df_run.groupBy(grpby_list)
                .agg(countDistinct('WAFER_ID').alias('wafer_count'),
                     countDistinct('WAFER_ID', when(df_run['label'] == 0, 1)).alias('good_num'),
                     countDistinct('WAFER_ID', when(df_run['label'] == 1, 1)).alias('bad_num'))
                .orderBy('bad_num', ascending=False))

        # 单站点+单腔室的情况
        if grps.count() == 1:
            return grps
        else:
            grps = grps.filter(grps['bad_num'] > 0)
            window_sep = Window().orderBy(col("bad_num").desc())
            ranked_df = grps.withColumn("rank", rank().over(window_sep))
            grpss = ranked_df.filter(col("rank") <= 10).drop("rank")
            return grpss
    except Exception as e:
        return None

In [8]:
df_run = _pre_process(df1)
print(df_run.count())

550896


In [37]:
grpby_list = ['OPER_NO', 'TOOL_NAME']
common_res = commonality_analysis(df_run, grpby_list)
common_res.show()

+--------+---------+-----------+--------+-------+
| OPER_NO|TOOL_NAME|wafer_count|good_num|bad_num|
+--------+---------+-----------+--------+-------+
|1F.EEK10|EKT72_PM1|       3892|    1207|   2685|
+--------+---------+-----------+--------+-------+



In [38]:
############################################################################
#################################获取样本数据#########################
############################################################################
def get_data_list(common_res, grpby_list, big_or_small='big'):
    """
    param common_res: 共性分析后的结果, 按照大样本或者小样本条件筛选出组合
    param grpby_list: 按照PRODG1+OPER_NO+TOOL_NAME分组或OPER_NO+TOOL_NAME分组
    param big_or_small: big或者small
    return: 对应组合的字典形式, 包在一个大列表中
    """
    assert big_or_small in ['big', 'small'], "只能选择big或者small, 请检查拼写"
    try:
        if big_or_small == 'big':
            good_bad_grps = common_res.filter("good_num >= 3 AND bad_num >= 3")
        else:
            good_bad_grps = common_res.filter("bad_num >= 1 AND wafer_count >=2")
        good_bad_grps = good_bad_grps.orderBy(col("bad_num").desc(), col("wafer_count").desc(), col("good_num").desc()).limit(5)
        
        if 'PRODG1' in grpby_list:
            data_list = good_bad_grps['PRODG1', 'OPER_NO', 'TOOL_NAME'].collect()  
        else:
            data_list = good_bad_grps['OPER_NO', 'TOOL_NAME'].collect()
            
        data_dict_list = [row.asDict() for row in data_list]
        return data_dict_list
    except Exception as e:
        return None


def get_train_data(df_run, data_dict_list):
    """
    param df_run: 数据预处理后的数据
    param data_dict: 筛选后的字典结果
    return: 从原始数据中过滤出真正用来建模的组合数据
    """
    try:
        if len(data_dict_list[0]) == 3:
            prod, oper, tool = data_dict_list[0]['PRODG1'], data_dict_list[0]['OPER_NO'], data_dict_list[0]['TOOL_NAME']
            df_s = df_run.filter("PRODG1 == '{}' AND OPER_NO == '{}' AND TOOL_NAME == '{}'".format(prod, oper, tool))
            for i in range(1, len(data_dict_list)):
                prod, oper, tool = data_dict_list[i]['PRODG1'], data_dict_list[i]['OPER_NO'], data_dict_list[i]['TOOL_NAME']
                df_m = df_run.filter("PRODG1 == '{}' AND OPER_NO == '{}' and TOOL_NAME == '{}'".format(prod, oper, tool))
                df_s = df_s.union(df_m)
        else:
            oper, tool = data_dict_list[0]['OPER_NO'], data_dict_list[0]['TOOL_NAME']
            df_s = df_run.filter("OPER_NO == '{}' AND TOOL_NAME == '{}'".format(oper, tool))
            for i in range(1, len(data_dict_list)):
                oper, tool = data_dict_list[i]['OPER_NO'], data_dict_list[i]['TOOL_NAME']
                df_m = df_run.filter("OPER_NO == '{}' and TOOL_NAME == '{}'".format(oper, tool))
                df_s = df_s.union(df_m)
        return df_s
    except Exception as e:
        return None

In [39]:
data_dict_list_bs = get_data_list(common_res=common_res, grpby_list=grpby_list, big_or_small='big')
data_dict_list_bs

[{'OPER_NO': '1F.EEK10', 'TOOL_NAME': 'EKT72_PM1'}]

In [40]:
df_run_bs = get_train_data(df_run=df_run, data_dict_list=data_dict_list_bs)
print(df_run_bs.count())

268064


In [41]:
############################################################################
#########################获取传入的整个数据中的所有bad_wafer个数####################
############################################################################
def get_all_bad_wafer_num(df):
    """
    param df: 筛选后的数据
    return: 数据中所有bad_wafer的数量
    """
    return df.filter("label == 1").select('WAFER_ID').distinct().count()

In [42]:
bad_wafer_num_big_sample = get_all_bad_wafer_num(df_run_bs)
bad_wafer_num_big_sample

2685

In [53]:
############################################################################
###########################对有good和bad的数据，建模############################
############################################################################
def get_pivot_table(df, by):
    if len(by) == 3:
        df_pivot = df.dropna(axis=0).pivot_table(index=['WAFER_ID', 'label'], 
                                                     columns=['OPER_NO', 'TOOL_NAME', 'parametric_name', 'PRODG1'],
                                                     values=['STATISTIC_RESULT'])
    else:
        df_pivot = df.dropna(axis=0).pivot_table(index=['WAFER_ID', 'label'], 
                                                     columns=['OPER_NO', 'TOOL_NAME', 'parametric_name'],
                                                     values=['STATISTIC_RESULT'])
    df_pivot.columns = df_pivot.columns.map('#'.join)
    df_pivot = df_pivot.fillna(df_pivot.mean()).reset_index(drop=False)
    return df_pivot



def fit_rf_big_sample(df, by):
    """
    param df: 大样本组合的数据
    param by: 分组字段
    return: RandomForest建模后的结果
    """
    try:
        schema_all = StructType([
            StructField("PRODG1", StringType(), True),
            StructField("OPER_NO", StringType(), True),
            StructField("TOOL_NAME", StringType(), True),
            StructField("bad_wafer", IntegerType(), True),
            StructField("roc_auc_score", FloatType(), True),
            StructField("features", StringType(), True),
            StructField("importance", FloatType(), True)])

        @pandas_udf(returnType=schema_all, functionType=PandasUDFType.GROUPED_MAP)
        def get_model_result(df_run):
            df_pivot = get_pivot_table(df=df_run, by=by)

            # 定义自变量和因变量
            X_train = df_pivot[df_pivot.columns.difference(['WAFER_ID', 'label']).tolist()]
            y_train = df_pivot[['label']]

            z_ratio = y_train.value_counts(normalize=True)
            good_ratio = z_ratio[0]
            bad_ratio = z_ratio[1]
            if abs(good_ratio - bad_ratio) > 0.7:
                undersampler = ClusterCentroids(random_state=101)
                X_train, y_train = undersampler.fit_resample(X_train, y_train)

            # 网格搜索
            pipe = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='constant', fill_value=-999)),
                ('scaler', StandardScaler()),
                ('model', RandomForestClassifier())])
            param_grid = {'model__n_estimators': [*range(50, 100, 10)],
                          'model__max_depth': [*range(10, 50, 10)]}
            grid = GridSearchCV(estimator=pipe, scoring='roc_auc', param_grid=param_grid, cv=3, n_jobs=-1)
            grid.fit(X_train.values, y_train.values.ravel())
            roc_auc_score_ = grid.best_score_

            # 特征重要度、结果汇总
            small_importance_res = pd.DataFrame({
                'features': X_train.columns,
                'importance': grid.best_estimator_.steps[2][1].feature_importances_}).sort_values(by='importance',
                                                                                                  ascending=False)
            if len(by) == 3:
                small_sample_res = pd.DataFrame({
                    'PRODG1': df_run['PRODG1'].unique(),
                    'OPER_NO': df_run['OPER_NO'].unique(),
                    'TOOL_NAME': df_run['TOOL_NAME'].unique(),
                    'bad_wafer': sum(df_pivot['label']),
                    'roc_auc_score': roc_auc_score_})
            else:
                PRODG1 = 'grplen2'
                small_sample_res = pd.DataFrame({
                    'PRODG1': PRODG1,
                    'OPER_NO': df_run['OPER_NO'].unique(),
                    'TOOL_NAME': df_run['TOOL_NAME'].unique(),
                    'bad_wafer': sum(df_pivot['label']),
                    'roc_auc_score': roc_auc_score_})
            return pd.concat([small_importance_res, small_sample_res])
        return df.groupby(by).apply(get_model_result)
    except Exception as e:
        return None

In [54]:
res = fit_rf_big_sample(df=df_run_bs, by=grpby_list)
res.show()

+------+-------+---------+---------+-------------+--------------------+-----------+
|PRODG1|OPER_NO|TOOL_NAME|bad_wafer|roc_auc_score|            features| importance|
+------+-------+---------+---------+-------------+--------------------+-----------+
|  null|   null|     null|     null|         null|STATISTIC_RESULT#...| 0.09229536|
|  null|   null|     null|     null|         null|STATISTIC_RESULT#...| 0.08904863|
|  null|   null|     null|     null|         null|STATISTIC_RESULT#...|0.076730885|
|  null|   null|     null|     null|         null|STATISTIC_RESULT#...| 0.07044043|
|  null|   null|     null|     null|         null|STATISTIC_RESULT#...| 0.06887801|
|  null|   null|     null|     null|         null|STATISTIC_RESULT#...|0.067021884|
|  null|   null|     null|     null|         null|STATISTIC_RESULT#...|0.061975103|
|  null|   null|     null|     null|         null|STATISTIC_RESULT#...|0.061536837|
|  null|   null|     null|     null|         null|STATISTIC_RESULT#...|0.030

In [55]:
res.toPandas()[['features']].iloc[0].values

array(['STATISTIC_RESULT#1F.EEK10#EKT72_PM1#LO_RF_VPP#STEP2_MEANDIFF#MEAN'],
      dtype=object)

In [46]:
#####################################################################################
#########################对有good和bad的建模后的结果进行整合###############################
#####################################################################################
def split_score_big_sample(df, by):
    """
    param df: RandomForest建模后的结果
    param by: 分组字段
    return: roc_auc分数结果
    """
    try:
        schema_all = StructType([StructField("PRODG1", StringType(), True),
                                 StructField("OPER_NO", StringType(), True),
                                 StructField("TOOL_NAME", StringType(), True),
                                 StructField("bad_wafer", IntegerType(), True),
                                 StructField("roc_auc_score", FloatType(), True)])

        @pandas_udf(returnType=schema_all, functionType=PandasUDFType.GROUPED_MAP)
        def get_result(model_results):
            sample_res = model_results[['PRODG1', 'OPER_NO', 'TOOL_NAME', 'bad_wafer', 'roc_auc_score']].dropna(axis=0)
            sample_res = sample_res[sample_res['roc_auc_score'] > 0.6]
            return sample_res
        return df.groupby(by).apply(get_result)
    except Exception as e:
        return None

In [47]:
s_res = split_score_big_sample(df=res, by=['PRODG1', 'OPER_NO', 'TOOL_NAME'])
s_res.show()

/usr/local/spark/python/pyspark/sql/pandas/group_ops.py:98: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


+-------+--------+---------+---------+-------------+
| PRODG1| OPER_NO|TOOL_NAME|bad_wafer|roc_auc_score|
+-------+--------+---------+---------+-------------+
|grplen2|1F.EEK10|EKT72_PM1|     2685|   0.92144805|
+-------+--------+---------+---------+-------------+



In [48]:
def split_features(df, index) -> str:
    """
    param df: RandomForest建模后的feature_importance_table
    param index: 顺序值
    return: 字段属性值
    """
    try:
        return df['features'].apply(lambda x: x.split('#')[index])
    except Exception as e:
        return None


def get_split_feature_importance_table(df, by):
    """
    param df: RandomForest建模后的feature_importance_table
    param by: OPER_NO+TOOL_NAME+PRODG1或者OPER_NO+TOOL_NAME
    return: 分裂features后的表
    """
    try:
        df['STATISTIC_RESULT'] = split_features(df, 0)
        df['OPER_NO'] = split_features(df, 1)
        df['TOOL_NAME'] = split_features(df, 2)
        df['parametric_name'] = split_features(df, 3)
        df['step'] = split_features(df, 4)
        df['stats'] = split_features(df, 5)

        if 'PRODG1' in by:
            df['PRODG1'] = split_features(df, 6)
        else:
            df = df.assign(PRODG1 = 'grplen2')

        df = df.drop(['features', 'STATISTIC_RESULT'], axis=1).reset_index(drop=True)
        return df
    except Exception as e:
        return None


def add_feature_stats(df):
    """
    param df: 经过处理后的feature_importance_table
    return: 新增一列，含有参数的所有统计特征:feature_stats
    """
    try:
        feature_stats = df.groupby(['PRODG1', 'OPER_NO', 'TOOL_NAME', 'parametric_name', 'step'])['stats'].unique().reset_index()
        feature_stats['stats'] = [feature_stats['stats'].iloc[i].tolist() for i in range(len(feature_stats))]
        feature_stats['stats'] = feature_stats['stats'].apply(lambda x: "#".join(x))
        feature_stats = feature_stats.assign(parametric_name=lambda x: x['parametric_name']+str('#')+x['step']).drop('step', axis=1)
        return feature_stats
    except Exception as e:
        return None

In [49]:
def split_calculate_features_big_sample(df, by):
    """
    param df: RandomForest建模后的结果
    param by: 分组字段
    return: features和importance结果
    """
    try:
        schema_all = StructType([
            StructField("PRODG1", StringType(), True),
            StructField("OPER_NO", StringType(), True),
            StructField("TOOL_NAME", StringType(), True),
            StructField("parametric_name", StringType(), True),
            StructField("importance", FloatType(), True),
            StructField("stats", StringType(), True)])

        @pandas_udf(returnType=schema_all, functionType=PandasUDFType.GROUPED_MAP)
        def get_result(model_results):
            # 先从随机森林的模型结果中取出包含features和importance的dataframe
            feature_importance_table = model_results[['features', 'importance']].dropna(axis=0)
            # 分裂features
            feature_importance_res_split = get_split_feature_importance_table(feature_importance_table, by)

            # 去除importance为0的组合
            feature_importance_res_split_drop = feature_importance_res_split.query("importance > 0").reset_index(drop=True)

            # 取每一种组合结果的前60%
            feature_importance_res_split_nlargest = (feature_importance_res_split_drop.groupby(by=['PRODG1', 'OPER_NO', 'TOOL_NAME'])
                                                .apply(lambda x: x.nlargest(int(x.shape[0]*0.6), 'importance') if x.shape[0]>1 else x.nlargest(int(x.shape[0]*1), 'importance'))
                                                .reset_index(drop=True))
            # 新增一列，含有参数的所有统计特征:feature_stats
            feature_stats = add_feature_stats(feature_importance_res_split_drop)

            # 对同一种组合里的同一个参数进行求和:feature_importance_groupby
            feature_importance_groupby = (feature_importance_res_split_nlargest.groupby(['PRODG1', 'OPER_NO', 'TOOL_NAME',
                                                                'parametric_name', 'step'])['importance'].sum().reset_index())
            feature_importance_groupby = feature_importance_groupby.assign(parametric_name=lambda x: x['parametric_name'] + str('#') + x['step']).drop('step', axis=1)

            # feature_stats和feature_importance_groupby连接
            grpby_stats = pd.merge(feature_stats, feature_importance_groupby, on=['PRODG1', 'OPER_NO', 'TOOL_NAME', 'parametric_name']).dropna().reset_index(drop=True)
            return grpby_stats
        return df.groupby(by).apply(get_result)
    except Exception as e:
        return None

In [50]:
f_res = split_calculate_features_big_sample(df=res, by=grpby_list)
f_res.show()

+-------+--------+---------+--------------------+------------+----------+
| PRODG1| OPER_NO|TOOL_NAME|     parametric_name|  importance|     stats|
+-------+--------+---------+--------------------+------------+----------+
|grplen2|1F.EEK10|EKT72_PM1|APC_POSITION#AOTU...| 0.020515786|MEAN#RANGE|
|grplen2|1F.EEK10|EKT72_PM1|AUTO_CHECK_LEAK_R...| 0.016150674|      MEAN|
|grplen2|1F.EEK10|EKT72_PM1|BOTTOMFLOWRATE#AO...|0.0058746045|      MEAN|
|grplen2|1F.EEK10|EKT72_PM1|CENTER_GAS_PRESSU...|  0.08329593|      MEAN|
|grplen2|1F.EEK10|EKT72_PM1|CENTER_GAS_PRESSU...|  3.65964E-4|      MEAN|
|grplen2|1F.EEK10|EKT72_PM1|CHAMBER_PRESSURE#...| 0.013456499|RANGE#MEAN|
|grplen2|1F.EEK10|EKT72_PM1|CHAMBER_PRESSURE#...| 0.001026323|RANGE#MEAN|
|grplen2|1F.EEK10|EKT72_PM1|CHAMBER_PRESSURE#...|3.7905315E-4|RANGE#MEAN|
|grplen2|1F.EEK10|EKT72_PM1|EDGE_GAS_PRESSURE...|  0.04766397|      MEAN|
|grplen2|1F.EEK10|EKT72_PM1|EDGE_HE_FLOW#AOTU...| 0.034963492|      MEAN|
|grplen2|1F.EEK10|EKT72_PM1|EDGE_HE_FL

In [51]:
def get_finall_results_big_sample(s_res, f_res, bad_wafer_num):
    """
    param s_res: roc_auc分数结果
    param f_res: features和importance结果
    param bad_wafer_num: 数据中所有bad_wafer的数量
    return: 最后的建模结果
    """
    try:
        # feature_importance_groupby和sample_res连接
        roc_auc_score_all = s_res.agg({"roc_auc_score": "sum"}).collect()[0][0]
        s_res = s_res.withColumn("roc_auc_score_ratio", col("roc_auc_score")/roc_auc_score_all)
        s_res = s_res.withColumn("bad_ratio", col("bad_wafer") / bad_wafer_num)

        df_merge = s_res.join(f_res, on=['PRODG1', 'OPER_NO', 'TOOL_NAME'], how='left')
        df_merge = df_merge.withColumn('weight_original', col('roc_auc_score_ratio') * col('bad_ratio') * col('importance'))

        # 最后再次进行一次归一化
        weight_all = df_merge.agg({"weight_original": "sum"}).collect()[0][0]
        df_merge = df_merge.withColumn("weight", col("weight_original") / weight_all)

        df_merge = df_merge.select(['PRODG1', 'OPER_NO', 'TOOL_NAME',
                                    'parametric_name', 'weight', 'stats']).orderBy('weight', ascending=False)
        return df_merge
    except Exception as e:
        return None

In [52]:
model_res_bs = get_finall_results_big_sample(s_res=s_res, f_res=f_res, bad_wafer_num=bad_wafer_num_big_sample)
model_res_bs.show()

+-------+--------+---------+--------------------+--------------------+----------+
| PRODG1| OPER_NO|TOOL_NAME|     parametric_name|              weight|     stats|
+-------+--------+---------+--------------------+--------------------+----------+
|grplen2|1F.EEK10|EKT72_PM1|LO_RF_VPP#AOTU_ST...|  0.1380923275625163|MEAN#SLOPE|
|grplen2|1F.EEK10|EKT72_PM1|ESC_CURRENT#AOTU_...| 0.11275109016774493|  MEAN#MAX|
|grplen2|1F.EEK10|EKT72_PM1|FLOWSPLITEDGE#AOT...| 0.08739145393710289|      MEAN|
|grplen2|1F.EEK10|EKT72_PM1|LO_RF_VPP#STEP2_M...|  0.0687084866149236|      MEAN|
|grplen2|1F.EEK10|EKT72_PM1|LO_RF_VPP#STEP2_MINI| 0.06761170739685284|      MEAN|
|grplen2|1F.EEK10|EKT72_PM1|CENTER_GAS_PRESSU...| 0.06206230109582653|      MEAN|
|grplen2|1F.EEK10|EKT72_PM1|FLOWSPLITCENTER#A...| 0.05986446560566939|      MEAN|
|grplen2|1F.EEK10|EKT72_PM1|LO_RF_REF_POWER#A...|0.044597637602787975|  MEAN#MAX|
|grplen2|1F.EEK10|EKT72_PM1|EDGE_GAS_PRESSURE...| 0.03774605219229366|      MEAN|
|grplen2|1F.EEK1

In [36]:
model_res_bs = get_finall_results_big_sample(s_res=s_res, f_res=f_res, bad_wafer_num=bad_wafer_num_big_sample)
model_res_bs.show()

+--------+--------+---------+--------------------+--------------------+----------+
|  PRODG1| OPER_NO|TOOL_NAME|     parametric_name|              weight|     stats|
+--------+--------+---------+--------------------+--------------------+----------+
|L11CD02A|1F.EEK10|EKT72_PM1|LO_RF_VPP#STEP2_M...| 0.13372307777728373|      MEAN|
|L11CD02A|1F.EEK10|EKT72_PM1|LO_RF_VPP#AOTU_ST...| 0.12585516985139056|MEAN#SLOPE|
|L11CD02A|1F.EEK10|EKT72_PM1|LO_RF_VPP#STEP2_MINI| 0.09897470007721072|      MEAN|
|L11CD02A|1F.EEK10|EKT72_PM1|PROCESS_GAS_8_O2#...| 0.06904437281416093|      MEAN|
|L11CD02A|1F.EEK10|EKT72_PM1|LOWER_TEMPERATURE...| 0.05277240244502971|      MEAN|
|L11CD02A|1F.EEK10|EKT72_PM1|LO_RF_REF_POWER#A...| 0.05168048646062583|  MEAN#MAX|
|L11CD02A|1F.EEK10|EKT72_PM1|LO_C1_VAR_CAPACIT...| 0.04450558511062986|     SLOPE|
|L11CD02A|1F.EEK10|EKT72_PM1|CHAMBER_PRESSURE#...|0.031448298017748656|RANGE#MEAN|
|L11CD02A|1F.EEK10|EKT72_PM1|CENTER_GAS_PRESSU...|0.030208098282600737|      MEAN|
|L11

----

In [86]:
# feature_importance_res_split1 = pd.DataFrame(
#     {'PRODG1': ['grplen2', 'grplen2', 'grplen2', 'grplen2', 'grplen2'],
#     'TOOL_NAME': ['PBT01_CGHA_4-14', 'PBT01_CGHA_4-14', 'PBT01_CGHA_4-14', 'PBT01_CGHA_4-13', 'PBT01_CGHA_4-13'],
#     'OPER_NO': ['1V.PPB10', '1V.PPB10', '1V.PPB10', '1V.PPB10', '1V.PPB10'],
#     'parametric_name': ['PLATE_TEMP', 'PLATE_TEMP', 'PLATE_TEMP', 'PLATE_TEMP', 'PLATE_TEMP'],
#     'importance': [0.7,0.2,0.1,0.9,0.1]},
# )
# feature_importance_res_split1

In [85]:
# # 取每一种组合结果的前60%
# feature_importance_res_split_nlargest = (feature_importance_res_split1.groupby(by=['PRODG1', 'OPER_NO', 'TOOL_NAME'])
#     .apply(lambda x: x.nlargest(int(x.shape[0]*1), 'importance') if x.shape[0]>1 else x.nlargest(int(x.shape[0]*1), 'importance'))
#     .reset_index(drop=True))
# feature_importance_res_split_nlargest

-----------

In [93]:
#####################################################################################
#############################将建模后的结果增加特定的列####################################
#####################################################################################
def add_certain_column(df, by, request_id):
    """
    param df: 最后的建模结果
    param by: 分组字段, 手动增加一列add
    param request_id: 传入的request_id
    return: 最后的建模结果增加特定的列
    """
    try:
        schema_all = StructType([
            StructField("PRODG1", StringType(), True),
            StructField("OPER_NO", StringType(), True),
            StructField("TOOL_NAME", StringType(), True),
            StructField("stats", StringType(), True),
            StructField("parametric_name", StringType(), True),
            StructField("weight", FloatType(), True),
            StructField("request_id", StringType(), True),
            StructField("weight_percent", FloatType(), True),
            StructField("index_no", IntegerType(), True)])

        @pandas_udf(returnType=schema_all, functionType=PandasUDFType.GROUPED_MAP)
        def get_result(final_res):
            final_res['weight'] = final_res['weight'].astype(float)
            final_res = final_res.query("weight > 0")
            final_res['request_id'] = request_id
            final_res['weight_percent'] = final_res['weight'] * 100
            final_res = final_res.sort_values('weight', ascending=False)
            final_res['index_no'] = [i + 1 for i in range(len(final_res))]
            final_res = final_res.drop('add', axis=1)
            # final_res['parametric_name'] = final_res['parametric_name'].str.replace("_", "+")
            final_res['PRODG1'] = final_res['PRODG1'].apply(lambda x: None if x == 'grplen2' else x)
            return final_res
        return df.groupby(by).apply(get_result)
    except Exception as e:
        return None

In [95]:
request_id = 'fdsf'
final_res_bs = model_res_bs.withColumn('add', lit(0))
final_res_add_columns = add_certain_column(df=final_res_bs, by='add', request_id=request_id)
final_res_add_columns.show()

/usr/local/spark/python/pyspark/sql/pandas/group_ops.py:98: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


+------+--------+---------------+-----+--------------------+------+----------+--------------+--------+
|PRODG1| OPER_NO|      TOOL_NAME|stats|     parametric_name|weight|request_id|weight_percent|index_no|
+------+--------+---------------+-----+--------------------+------+----------+--------------+--------+
|  null|1V.PPB10|PBT01_CGHA_4-14| MEAN|PLATE_TEMP#HDB205...|   1.0|      fdsf|         100.0|       1|
+------+--------+---------------+-----+--------------------+------+----------+--------------+--------+



In [ ]:
#####################################################################################
################################将最后的结果写回数据库####################################
#####################################################################################
def doris_stream_load_from_df(df, engine, table, is_json=True, chunksize=100000, partitions=None):
    engine_url = engine.url
    url = 'http://%s:18030/api/%s/%s/_stream_load' % (engine_url.host, engine_url.database, table)

    format_str = 'csv' if not is_json else 'json'
    headers = {
        'Content-Type': 'text/plain; charset=UTF-8',
        'format': format_str,
        'Expect': '100-continue'
    }
    if is_json:
        headers['strip_outer_array'] = 'true'
        headers['read_json_by_line'] = 'true'
    else:
        headers['column_separator'] = '@'
    
    if partitions:
        headers['partitions'] = partitions
    
    auth = requests.auth.HTTPBasicAuth(engine_url.username, engine_url.password)
    session = requests.sessions.Session()
    session.should_strip_auth = lambda old_url, new_url: False
    
    l = len(df)
    if l > 0:
        if chunksize and chunksize < l:
            batches = l // chunksize
            if l % chunksize > 0:
                batches += 1
            for i in range(batches):
                si = i * chunksize
                ei = min(si + chunksize, l)
                sub = df[si:ei]
                do_doris_stream_load_from_df(sub, session, url, headers, auth, is_json)
        else:
            do_doris_stream_load_from_df(df, session, url, headers, auth, is_json)


def do_doris_stream_load_from_df(df, session, url, headers, auth, is_json=False):
    data = df.to_csv(header=False, index=False, sep='@') if not is_json else df.to_json(orient='records', date_format='iso')
    #print(data)
    
    resp = session.request(
        'PUT',
        url = url,
        data=data.encode('utf-8'),
        headers=headers,
        auth=auth
    )
    print(resp.reason, resp.text)
    check_stream_load_response(resp.text)


def check_stream_load_response(resp_text):
    resp = json.loads(resp_text)
    if resp['Status'] not in ["Success", "Publish Timeout"]:
        raise Exception(resp['Message'])